In [1]:
!pip install Tashaphyne


In [2]:
!pip install isri

ERROR: Could not find a version that satisfies the requirement isri (from versions: none)
ERROR: No matching distribution found for isri


In [3]:
import pandas as pd
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tashaphyne.stemming import ArabicLightStemmer
# Download NLTK resources if not already available
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Musae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Musae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:

# Initialize the Arabic Light Stemmer
ArListem = ArabicLightStemmer()


In [5]:
# Define Arabic and English punctuation
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

# Normalization function for Arabic
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ئ", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text



In [6]:
# Define Arabic stopwords
arabic_stopwords = set(stopwords.words('arabic'))

# Preprocessing function to clean and stem text data
def preprocess_text(text):
    # Normalize text
    text = normalize_arabic(text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove punctuation and stopwords, convert to lowercase
    cleaned_tokens = [word.lower() for word in tokens if word not in punctuations_list and word not in arabic_stopwords]
    # Stem each token using Arabic Light Stemmer
    stemmed_tokens = [ArListem.light_stem(word) or ArListem.get_root() for word in cleaned_tokens]
    return ' '.join(stemmed_tokens)

In [7]:
# Load the dataset
df = pd.read_csv('C:\\Users\\Musae\\Documents\\GitHub-REPOs\\NLP-Project\\data\\ar_reviews_100k.tsv', sep='\t')

# Apply preprocessing to the text column
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Display the first 10 entries of the cleaned text
print(df[['text', 'cleaned_text']].head(10))

                                                text  \
0  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...   
1  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...   
2  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...   
3  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...   
4  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...   
5  أسلوب الكاتب رائع جدا و عميق جدا، قرأته عدة مر...   
6  استثنائي. الهدوء في الجناح مع مسبح. عدم وجود ع...   
7  الكتاب هو السيرة الذاتية للحداثة في المملكة بل...   
8       من أجمل ما قرأت.. رواية تستحق القراءة فعلا..   
9  بشكل عام جيده .. . التجاوب جيد جدا من قبل موظف...   

                                        cleaned_text  
0              ممتاز وعا نظافه موقع تجهيز شاطيء مطعم  
1  حد سباب جاح امار ان شخص دوله يعشق راب حب امار ...  
2  هادف .. قو نقل صخب شوارع قاهره لي هدوء جبال شي...  
3  خلص .. مبديي لي مس بهار زي فيل ازرق ميقراش حس ...  
4   ياس جلوري جزء تجز دب دق متكامل خدم مريح فسيا وجد  
5  سلوب كاتب رايع جد عميق جدا، قر عد مر نت طالب م... 

In [8]:
positive_keywords = {
    'ممتاز', 'جيد', 'رائع', 'سعيد', 'لذيذ', 'مبهج', 'فرح', 'استثنائي', 'جميل', 'محبب', 'ممتع',
    'مذهل', 'مريح', 'راض', 'أحب', 'استمتع', 'مفاجأة', 'مميز', 'لطيف', 'مرح', 'معجزة',
    'ملهم', 'أسعد', 'خيالي', 'مذهل', 'فريد', 'هائل', 'راقي', 'أنيق', 'بهجة', 'مفيد',
    'قيمة', 'بسيط', 'ناجح', 'موفق', 'مشجع', 'حسن', 'ظريف', 'محبوب', 'مبهر', 'إيجابي',
    'تفاؤل', 'إعجاب', 'ممتن', 'شجاع', 'آمن', 'مثالي'
}

negative_keywords = {
    'سيء', 'مخيب', 'حزين', 'مؤلم', 'كريه', 'قبيح', 'فشل', 'محبط', 'بشع', 'فظيع', 'مزعج', 
    'مروع', 'أسوأ', 'كره', 'كارثة', 'رعب', 'كئيب', 'مزعزع', 'اكتئاب', 'بائس', 'معقد', 
    'إحباط', 'تعب', 'مضجر', 'ممل', 'فضيحة', 'سلبي', 'كاذب', 'فظاظة', 'احتيال', 'احراج',
    'بشع', 'تعيس', 'مستاء', 'مروع', 'مشؤوم', 'عداء', 'مزري', 'عنيف', 'غير كاف', 'صادق',
    'ضعيف', 'متشائم', 'غاضب', 'مقبول', 'مروع'
}


In [9]:
# Rule-based classification function
def classify_review(text):
    # Tokenize the input text
    words = set(text.split())
    # Initialize flags to check for keyword presence
    has_positive = bool(words & positive_keywords)
    has_negative = bool(words & negative_keywords)

    # Basic rule-based classification
    if has_positive and not has_negative:
        return "Positive"
    elif has_negative and not has_positive:
        return "Negative"
    else:
        return "Mixed"

# Apply classification to cleaned text
df['predicted_label'] = df['cleaned_text'].apply(classify_review)

# Display the original, cleaned, and predicted labels for the first 10 reviews
print(df[['text', 'cleaned_text', 'label', 'predicted_label']].head(10))

# Evaluate the classifier
predicted_counts = df['predicted_label'].value_counts()

print("\nPredicted sentiment counts:")
print(predicted_counts)


                                                text  \
0  ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...   
1  أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...   
2  هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...   
3  خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...   
4  ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...   
5  أسلوب الكاتب رائع جدا و عميق جدا، قرأته عدة مر...   
6  استثنائي. الهدوء في الجناح مع مسبح. عدم وجود ع...   
7  الكتاب هو السيرة الذاتية للحداثة في المملكة بل...   
8       من أجمل ما قرأت.. رواية تستحق القراءة فعلا..   
9  بشكل عام جيده .. . التجاوب جيد جدا من قبل موظف...   

                                        cleaned_text     label predicted_label  
0              ممتاز وعا نظافه موقع تجهيز شاطيء مطعم  Positive        Positive  
1  حد سباب جاح امار ان شخص دوله يعشق راب حب امار ...  Positive           Mixed  
2  هادف .. قو نقل صخب شوارع قاهره لي هدوء جبال شي...  Positive           Mixed  
3  خلص .. مبديي لي مس بهار زي فيل ازرق ميقراش حس ...  Posit